In [115]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'c:\\Users\\perry\\Desktop\\Research\\antibodyGPT\\model_v2\\ab_curation\\atlas_8365-AA'

In [116]:
df = pd.read_csv('./Atlas_systems_wide_data_IgG_noDengue_23-06-02.csv', index_col=0)

In [117]:
df= df[df['Sample'] != '8365-19']

In [118]:
seq_df = df[['SEQUENCE_INPUT.H', 'SEQUENCE_INPUT.L']]

In [119]:
def convert_original_list(input_csv, output_fasta_name): # 
    output = input_csv.copy()

    output.index = '>' + output.index

    hchains = pd.DataFrame(index=output.index)
    hchains['Seq'] = output['SEQUENCE_INPUT.H']
    hchains.index = hchains.index + ' HC'

    lchains = pd.DataFrame(index=output.index)
    lchains['Seq'] = output['SEQUENCE_INPUT.L']
    lchains.index = lchains.index + ' LC'

    output = pd.concat([hchains, lchains])


    path = output_fasta_name + '.fasta'

    with open(path, "w") as f:

        for i in range(len(output)):
            feature = i + 1
            header = output.iloc[i].name
            line1 = output.iloc[i][0]

            # break
            f.write(header + '\n')

            # print('line1 + '\n')
            f.write(line1 + '\n')

# convert_original_list(seq_df, 'atlas_systems_genLLM_nuc_seqs_24-03-04')

In [120]:
bin_cols = [i for i in df if '.bin' in i]
print(bin_cols)

['CoV_HKU1.bin', 'CoV_OC43.bin', 'D1.bin', 'D2.bin', 'D3.bin', 'D4.bin', 'H1_NC99.bin', 'H1_MI15.bin', 'H3_Perth.bin', 'H3_HK68.bin', 'H5_VN04.bin', 'H7.bin', 'H9.bin', 'H10.bin', 'RSV_A.bin', 'RSV_B.bin', 'MPV_A.bin', 'MPV_B.bin', 'PIV_3.bin', 'BG505_sc.bin', 'SARS-2.bin']


In [121]:
def get_binders(col):
    antigen = col.split('.bin')[0]
    binder_df = df[df[col] == 1][[(antigen + '.LSS'), antigen]].copy()
    binder_df.index = antigen + ':' + binder_df.index
    binder_df.columns = ['LSS', 'UMI']
    binder_df['antigen'] = antigen
    return binder_df

binned_df = df[bin_cols]
binders = pd.concat([get_binders(col) for col in binned_df])

In [122]:
print(binders.shape)
binders = binders[binders['LSS'] >= 2]
print(binders.shape)
binders = binders[binders['UMI'] >= 30]
print(binders.shape)


(2717, 3)
(1705, 3)
(1425, 3)


In [123]:
antigen_seqs = pd.read_csv('../antigen_sequences_LIBRA-seq_24-01-08.csv')
antigen_dict = dict(zip(antigen_seqs['LSS_name'], antigen_seqs['Seq']))

In [135]:
# Downloaded from plasmid here: https://www.addgene.org/154754/
hexapro = 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSPIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGPALQIPFPMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTPSALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQ'
print(len(hexapro))
antigen_dict['SARS-2.LSS'] = hexapro

1208


In [136]:
# Make col to map antigen sequence to 
binders['antigen_seq'] = binders['antigen'] + '.LSS'
binders['antigen_seq'] = binders['antigen_seq'].map(antigen_dict)

In [137]:
binders['antigen'].value_counts()

SARS-2      391
H1_MI15     183
MPV_A       178
H1_NC99     165
MPV_B       153
RSV_A       110
RSV_B        92
PIV_3        55
H5_VN04      27
H3_Perth     25
H7           18
H3_HK68      10
H9            8
CoV_HKU1      4
H10           4
CoV_OC43      2
Name: antigen, dtype: int64

In [139]:
binders[binders['antigen_seq'].isna()]['antigen'].value_counts()

H7          18
CoV_HKU1     4
H10          4
CoV_OC43     2
Name: antigen, dtype: int64

In [140]:
binders[~binders['antigen_seq'].isna()].shape

(1397, 4)

Loading in IMGT outputs

In [15]:
import os
os.listdir('all_systems_atlas_gen_LLM_IMGT')

['10_V-REGION-mutation-hotspots.txt',
 '11_Parameters.txt',
 '1_Summary.txt',
 '2_IMGT-gapped-nt-sequences.txt',
 '3_Nt-sequences.txt',
 '4_IMGT-gapped-AA-sequences.txt',
 '5_AA-sequences.txt',
 '6_Junction.txt',
 '7_V-REGION-mutation-and-AA-change-table.txt',
 '8_V-REGION-nt-mutation-statistics.txt',
 '9_V-REGION-AA-change-statistics.txt',
 'README.txt']

In [143]:
imgt_output = pd.read_csv('./all_systems_atlas_gen_LLM_IMGT/5_AA-sequences.txt', sep= '\t', index_col=0)

imgt_output.index = imgt_output['Sequence ID'].apply(lambda x: x.split('_')[0]).rename('BARCODE')
imgt_output['chain'] = imgt_output['Sequence ID'].apply(lambda x: x.split('_')[2])

In [144]:
imgt_output = pd.merge(imgt_output[imgt_output['chain'] == 'HC'], imgt_output[imgt_output['chain'] == 'LC'], left_index=True, right_index=True, suffixes=('.H', '.L'))

In [145]:
# imgt_output.to_csv('atlas_8365_binding_IMGT_annot_24-03-05.csv')

In [55]:
# input_seqs = imgt_output[['V-D-J-REGION.H', 'V-J-REGION.L']]

In [146]:
# Getting barcode for mapping in variable regions
binders['VH_AA'] = [i.split(':')[1].split('_')[0] for i in binders.index]
binders['VL_AA'] = [i.split(':')[1].split('_')[0] for i in binders.index]

binders['VH_AA'] = binders['VH_AA'].map(dict(zip(imgt_output.index, imgt_output['V-D-J-REGION.H'])))
binders['VL_AA'] = binders['VL_AA'].map(dict(zip(imgt_output.index, imgt_output['V-J-REGION.L'])))

In [152]:
seq_df = binders[~binders['antigen_seq'].isna()]

In [153]:
# seq_df.to_csv('atlas8365_binders_input_seq_v1_24-03-05.csv')